# MLRun Job Example

### Setup MLRun

In [1]:
import os
from os import path
from mlrun import set_environment, new_project, mlconf, code_to_function, mount_v3io, get_run_db

# Set the default environment configuration
project_name, artifact_path = set_environment(
    project="igz-onboarding",
    artifact_path="/User/artifacts/{{run.project}}/{{workflow.uid}}"
)

# Create project
project_path = path.abspath("project")
project = new_project(name=project_name, context=project_path)

print(f"Project name: {project_name}\nProject path: {project_path}")
print(f"Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}")

Project name: igz-onboarding
Project path: /User/igz_repos/igz-onboarding/project
Artifacts path: /User/artifacts/{{run.project}}/{{workflow.uid}}
MLRun DB path: http://mlrun-api:8080


### Build Docker Image

In [2]:
image = f"docker-registry.{os.getenv('IGZ_NAMESPACE_DOMAIN')}:80/new-image"
image

'docker-registry.default-tenant.app.us-sales-fs.iguazio-cd0.com:80/new-image'

In [3]:
# # Build Docker Image (only needs to be run once)
# build_image = new_function(name="build-image", kind="job")
# build_image.build_config(
#     image=image, base_image="mlrun/ml-models-gpu", commands=["pip install v3io_frames"]
# )
# build_image.deploy(with_mlrun=False)

### Define MLRun Function

In [4]:
# Create MLRun function from Python file
fn = code_to_function(
    name="training-mount-demo",
    project=project_name,
    filename="mlrun_mount_fn.py",
    handler="handler",
    kind='job',
    image="mlrun/mlrun"
)

# Mount current directory to /src in container
fn.apply(
    mount_v3io(
        name="src",
        remote="users/nick/igz_repos/igz-onboarding",
        mount_path="/src"
    )
)

### Run MLRun Function on Cluster

In [5]:
run = fn.run(params={'name' : "testing"}, artifact_path=artifact_path)

> 2021-05-13 00:04:02,606 [info] starting run training-demo-handler uid=d272ca2e16024c40a2c96121a9024560 DB=http://mlrun-api:8080
> 2021-05-13 00:04:02,781 [info] Job is running in the background, pod: training-demo-handler-ptlsj
Hello from mymodule!
Hi from print_hi.py
> 2021-05-13 00:04:07,400 [info] *****Hello from test.py, testing!*****
> 2021-05-13 00:04:07,435 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
igz-onboarding,...a9024560,0,May 13 00:04:07,completed,training-demo-handler,v3io_user=nickkind=jobowner=nickhost=training-demo-handler-ptlsj,,name=testing,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run d272ca2e16024c40a2c96121a9024560 --project igz-onboarding , !mlrun logs d272ca2e16024c40a2c96121a9024560 --project igz-onboarding
> 2021-05-13 00:04:08,988 [info] run executed, status=completed


### View Run Logs

In [6]:
db = get_run_db()
resp = db.watch_log(run.uid(), project=project_name)

Hello from mymodule!
Hi from print_hi.py
> 2021-05-13 00:04:07,400 [info] *****Hello from test.py, testing!*****
> 2021-05-13 00:04:07,435 [info] run executed, status=completed

